In [8]:
import os
import torch
import numpy as np
import torch.nn.functional as F

from conditional_rate_matching.data.dataloaders_utils import get_dataloaders_crm
from conditional_rate_matching.models.generative_models.crm import uniform_pair_x0_x1
from conditional_rate_matching.configs.configs_classes.config_crm import CRMConfig as ConditionalRateMatchingConfig

from conditional_rate_matching.models.generative_models.crm import (
    CRM
)

from conditional_rate_matching.configs.config_files import ExperimentFiles
from conditional_rate_matching.models.trainers.crm_trainer import CRMTrainer
from conditional_rate_matching.configs.config_files import get_experiment_dir
from conditional_rate_matching.configs.configs_classes.config_crm import CRMConfig

from conditional_rate_matching.utils.plots.images_plots import plot_sample
from conditional_rate_matching.models.pipelines.sdes_samplers.samplers import TauLeaping
from conditional_rate_matching.configs.experiments_configs.crm.crm_experiments_nist import experiment_nist

import pytest
from conditional_rate_matching.models.metrics.crm_path_metrics import conditional_bridge_marginal_probabilities_and_rates_path,conditional_bridge_images
from conditional_rate_matching.utils.plots.images_plots import mnist_grid,mnist_noise_bridge
from conditional_rate_matching import results_path
from conditional_rate_matching import plots_path
from conditional_rate_matching.models.metrics.fid_metrics import load_classifier
from matplotlib import pyplot as plt

In [39]:
##experiment_dir = os.path.join(results_path,"emnist_2_mnist_lenet5","emnist_2_mnist_lenet5","run")
#experiment_dir = os.path.join(results_path,"emnist_2_mnist_lenet5_OTPlanSampler","emnist_2_mnist_lenet5_OTPlanSampler","run")
#experiment_dir = os.path.join(results_path,"emnist_2_mnist_lenet5_UniformCoupling","emnist_2_mnist_lenet5_UniformCoupling","run")
experiment_dir = os.path.join(results_path,"emnist_2_mnist_lenet5_OTPlanSampler_1","emnist_2_mnist_lenet5_OTPlanSampler_1","run")

#experiment_dir = get_experiment_dir(experiment_name="prenzlauer_experiment",
#                                    experiment_type="crm",
#                                    experiment_indentifier="bridge_plot_mlp_mu_001")

label_to_see = 20
number_of_images_to_see = 20
steps_of_noise_to_see = 19

save_path = os.path.join(plots_path,"emnist_to_mnist_lenet5_Uniform_01.png")
device = torch.device("cpu")
crm = CRM(experiment_dir=experiment_dir, device=device)


number_of_source_labels = crm.config.data0.number_of_labels
number_of_target_labels = crm.config.data1.number_of_labels
classifier = load_classifier(crm.config.data1.dataset_name,device) #CLASSIFIES TARGET AT TIME 1

label_to_label_histograms = {label_source:torch.zeros(number_of_target_labels) for label_source in range(number_of_source_labels)}
source_label_numbers = {label_source:0. for label_source in range(number_of_source_labels)}

max_number_of_batches = 1
number_of_batches = 0

for databatch_0 in crm.dataloader_0.test():
    images_ = databatch_0[0]
    labels_ = databatch_0[1]

    #evolves from the whole batch then performs statistics
    crm.config.pipeline.num_intermediates = steps_of_noise_to_see
    x_f = crm.pipeline(100,train=True,x_0=images_)

    for label_to_see in range(number_of_source_labels):
        selected_index = labels_ == label_to_see
        selected_images = images_[selected_index]
        selected_target = x_f[selected_index]
        num_images_encountered = selected_images.size(0)
        
        if num_images_encountered > 0:
            source_label_numbers[label_to_see] += num_images_encountered # how many images of that label in source
            y = classifier(selected_target.view(-1,1,28,28))
            y = torch.argmax(y,dim=1)
            label_to_label_histograms[label_to_see] += F.one_hot(y,number_of_labels).sum(axis=0) # how many of the target images are encountered from that source

    number_of_batches+=1
    
    if number_of_batches >= max_number_of_batches:
        break
y y
    for source_label in range(number_of_source_labels):
        label_to_label_histograms[source_label] =  label_to_label_histograms[source_label]/source_label_numbers[source_label]

1000it [00:43, 22.81it/s]


In [ ]:
x_f, x_hist, ts = crm.pipeline(100,return_intermediaries=True,train=False,x_0=selected_images)
print(x_f.shape)
histograms_of_labels = torch.zeros(number_of_labels)

In [11]:
torch.argmax(classifier(x_f.view(-1,1,28,28)),dim=1)

tensor([9, 6, 7, 2, 3, 6, 1, 0, 2, 9, 1, 2, 8, 9, 3, 9, 1, 4, 3, 7, 9])